#### Observe that we can get news of a day with the link format:
http://news.ltn.com.tw/list/newspaper/politics/20181231
#### We need a list of date string in the date range we want.

In [1]:
from datetime import datetime, timedelta

start_date = "2018-07-01"
stop_date = "2018-12-31"

start = datetime.strptime(start_date, "%Y-%m-%d")
stop = datetime.strptime(stop_date, "%Y-%m-%d")

dates = list()
while start <= stop:
    dates.append(start.strftime('%Y%m%d'))
    start = start + timedelta(days=1)


In [2]:
import requests
from bs4 import BeautifulSoup as bs

In [3]:
def process_document(document, date):
    
    nodes = document.select('ul.list > li')
    data = list()

    for li in nodes:

        # check if is empty element
        if li.select_one('a') == None:
            continue

        # get link
        li_link = 'http://news.ltn.com.tw/' + li.select_one('a')['href']

        # request for document
        li_res = requests.get(li_link)
        li_doc = bs(li_res.text, 'lxml')

        # get date
        li_date = datetime.strptime(date, "%Y%m%d").strftime('%Y-%m-%d')

        #get title
        li_title = li.select_one('p').get_text()

        #get content
        li_content = ""
        for ele in li_doc.select('div.text > p'):
            li_content += ele.get_text()

        # append new row
        data.append({
            'date' : li_date,
            'title': li_title,
            'link' : li_link,
            'content' : li_content,
            'tags' : []
        })
    return data

In [5]:
cnt = 0
all_data = list()
for date in dates:
    print('start crawling :', date)
    res = requests.get('https://news.ltn.com.tw/list/newspaper/politics/' + date)
    doc = bs(res.text, 'lxml')
    data = process_document(doc, date)
    all_data += data
all_data = all_data[['date', 'title', 'link', 'content', 'tags']]

start crawling : 20180701
start crawling : 20180702
start crawling : 20180703
start crawling : 20180704
start crawling : 20180705
start crawling : 20180706
start crawling : 20180707
start crawling : 20180708
start crawling : 20180709
start crawling : 20180710
start crawling : 20180711
start crawling : 20180712
start crawling : 20180713
start crawling : 20180714
start crawling : 20180715
start crawling : 20180716
start crawling : 20180717
start crawling : 20180718
start crawling : 20180719
start crawling : 20180720
start crawling : 20180721
start crawling : 20180722
start crawling : 20180723
start crawling : 20180724
start crawling : 20180725
start crawling : 20180726
start crawling : 20180727
start crawling : 20180728
start crawling : 20180729
start crawling : 20180730
start crawling : 20180731
start crawling : 20180801
start crawling : 20180802
start crawling : 20180803
start crawling : 20180804
start crawling : 20180805
start crawling : 20180806
start crawling : 20180807
start crawli

TypeError: list indices must be integers or slices, not list

In [13]:
cols =
all_data = all_data[['date', 'title', 'link', 'content', 'tags']]


AttributeError: 'list' object has no attribute 'columns'

In [5]:
all_data[0:5]

[{'date': '2018-07-01',
  'title': '新南向救災合作 推動消防設備輸出',
  'link': 'http://news.ltn.com.tw/news/politics/paper/1213127',
  'content': '〔記者李欣芳／台北報導〕政府積極推動新南向政策，其中推動與新南向國家的救災合作已有腹案，行政院資深官員昨透露，政府除一方面推動與新南向目標國的救災訓練合作外，另方面也要促使台灣消防等救災設備與警報器等儀器的輸出，以協助我廠商爭取商機，目前並規畫與東南亞國家等救災訓練的合作，將以消防署南投竹山訓練中心為訓練基地。南投竹山的消防署訓練中心不僅是國內最大專供消防及災害防救使用的訓練基地，其規模也是全亞洲第一大，這個中心除提供消防、國軍、警大警專消防專業訓練外，也是各級政府災害防救演練及民間救難團體各項救難的訓練場所。資深官員表示，竹山訓練中心是全亞洲最大的災防訓練中心，台灣又擁有優秀的消防等救災人員，未來與新南向國家談妥合作後，對方的人員來台，就會在竹山訓練中心受訓，由我方協助訓練新南向國家的救災人員。據悉，行政院經貿談判辦公室衝刺新南向重點工作計畫，基於台灣地震頻仍、消防等緊急救災有豐富經驗，經貿辦主動提出構想，要推動與新南向國家的救災訓練等合作，目前包括東南亞地區在內，已有國家表達願意與台灣合作的興趣與意願。政府捐助預算成立的新南向智庫「台灣亞洲交流基金會」，有意推動人道救援論壇，由台灣與新南向國家共同探討災變的急難救助與防災議題。資深官員評估，政府推動救災訓練合作，對台灣與新南向國家而言，最終是彼此互惠關係，東南亞等國家可因此增進其災防與急難救助的能力，台灣則可輸出消防衣、滅火器、消防栓設備、自動灑水設備等消防設備與火警自動警報器等儀器，對台灣而言也有不少商機。還想看更多新聞嗎？歡迎下載自由時報APP，現在看新聞還能抽獎，共7萬個中獎機會等著你：iOS載點 https://goo.gl/Gc70RZAndroid載點 https://goo.gl/VJf3lv活動辦法： https://draw.ltn.com.tw/slot_v8/',
  'tags': []},
 {'date': '2018-07-01',
  'title': '竹山訓練中心 亞洲最大',
  'link': 'http://n

In [6]:
import pickle

with open('data/liberty_times.pkl', 'wb') as f:
    pickle.dump(all_data, f)

In [7]:
import pandas as pd
pd.DataFrame(all_data)[['date', 'title', 'link', 'content', 'tags']]

,date,title,link,content,tags
0,2018-07-01,新南向救災合作 推動消防設備輸出,http://news.ltn.com.tw/news/politics/paper/121...,〔記者李欣芳／台北報導〕政府積極推動新南向政策，其中推動與新南向國家的救災合作已有腹案，行政...,[]
1,2018-07-01,竹山訓練中心 亞洲最大,http://news.ltn.com.tw/news/politics/paper/121...,〔記者陳薏云／新北報導〕位於南投竹山的消防署訓練中心於二○一○年一月十九日啟用，是目前亞洲規...,[]
2,2018-07-01,9月正式開幕 我盼美高階官員來台 見證AIT新館,http://news.ltn.com.tw/news/politics/paper/121...,〔記者蘇永耀、彭琬馨／台北報導〕美國在台協會（ＡＩＴ）內湖新館已於六月落成，正式運作將至九月...,[]
3,2018-07-01,陳破空︰美中對決 台應掌握戰略機遇,http://news.ltn.com.tw/news/politics/paper/121...,〔記者呂伊萱／台北報導〕美國總統川普上任後，以非典型領導與強勢多變的談判作風「讓美國再次強大...,[]
4,2018-07-01,冷眼集》大大同情中國當局,http://news.ltn.com.tw/news/politics/paper/121...,記者鄒景雯／特稿最近有位被拒絕入境的中國記者，對於無法再回到台灣駐點的遭遇，表達了「有一點點...,[]
5,2018-07-01,加拿大國慶 本土社團喊「加航可恥」,http://news.ltn.com.tw/news/politics/paper/121...,〔記者彭琬馨／台北報導〕加拿大商會、加拿大駐台機構昨日在台北市客家文化主題公園舉辦加拿大國慶...,[]
6,2018-07-01,王金平號召捐血萬袋 立委挽袖自拍,http://news.ltn.com.tw/news/politics/paper/121...,國民黨立委、前立法院長王金平昨號召多個團體發起「萬袋熱血展愛心」的全台捐血活動，目標全台募集...,[]
7,2018-07-01,星期專論》台灣必須適應不斷變動的戰略環境,http://news.ltn.com.tw/news/politics/paper/121...,◎何瑞恩（Ryan Hass）美國總統川普與北韓領導人金正恩六月十二日在新加坡的歷史性高峰會...,[]
8,2018-07-01,一週大事（6月24日至30日）,http://news.ltn.com.tw/news/politics/paper/121...,國內：●立法院臨時會廿七日通過「財團法人法」，對政府捐助的財團法人進行高密度管理，對於原為政...,[]
9,2018-07-02,謠言終結站》軍改通知「習近平」？國防部斥捏造,http://news.ltn.com.tw/news/politics/paper/121...,軍方寄發退伍軍人退俸重新計算書，因先前作業疏失，導致有許多姓名、退階及地址錯置的情形，網路上...,[]
